In [28]:
import newspaper
import requests
from bs4 import BeautifulSoup
import psycopg2

In [7]:
blogs = ['https://foxnomad.com/blog/', 'https://www.cheapestdestinationsblog.com/', 'https://globalhelpswap.com/travel-blog/', 'http://www.matthew-woodward.com/blog/', 'http://www.oneikathetraveller.com/']

In [31]:
blog = newspaper.Source('http://www.oneikathetraveller.com/')
blog.build()
blog.size()

16

In [30]:
conn = psycopg2.connect(host='localhost', user='postgres', password='postgres', port=5432, database='blogscrapes')

In [31]:
cur = conn.cursor()

In [33]:
# This will get the links for all blogposts on foxnomad.com
pagelink = 'https://foxnomad.com/blog/'
while True:
    try:
        res = requests.get(pagelink)
        soup = BeautifulSoup(res.text)
        for post in soup.select('.post-title'):
            link = post.select('a')[0]['href']
            article = newspaper.Article(link, fetch_images=False)
            article.download()
            article.parse()
            article.nlp()
            thetext = article.text
            summary = article.summary
            sql = """
            INSERT INTO scrapes (url, text_data, summary) VALUES (%s, %s, %s);
            """
            cur.execute(sql, (link, thetext, summary))
            conn.commit()
        pagelink = soup.select('a.next')[0]['href']
    except IndexError:
        print('Seems like all has been scraped. Let\'s move on :\)')
        break

ArticleException: Article `download()` failed with HTTPSConnectionPool(host='foxnomad.com', port=443): Read timed out. (read timeout=7) on URL https://foxnomad.com/2006/10/08/youtubes-might-be-in-danger/

'https://foxnomad.com/blog/page/2/'

In [25]:
res = requests.get('https://foxnomad.com/blog/page/460/')
soup = BeautifulSoup(res.text)
soup.select('a.next')[0]['href']

IndexError: list index out of range